In [3]:
import pandas as pd
import numpy as np

In building a model for this data, first I am going to import the sets and look for missing variables. The missing variables will be filled with what makes sense in their context and given their correlations. Then I will do feature engineering to 1) make sure all variables are coded as the correct type and 2) scale variables by numer of people in the household and age where appropriate. Then I will subset down to the head of households to train my model, (because head of households are what is the important target), and finally I will run a number of models to categorize the households into their poverty classifications. 

In [4]:
train=pd.read_csv(r"C:\Users\owner\Documents\Junior\S2\Machine_Learning\HW4\train.csv")
test=pd.read_csv(r"C:\Users\owner\Documents\Junior\S2\Machine_Learning\HW4\test.csv")

In [5]:
print(test.shape)
print(train.shape)

(23856, 142)
(9557, 143)


I'm going to put my training and testing data in the same set so I can make sure all missing values are taken into account and that the two sets have the same number of columns in the end. 

In [6]:
ntrain=train.shape[0]
ntest=test.shape[0]
test['Target']=0
all_data = pd.concat((train, test)).reset_index(drop=True)

In [7]:
all_data.head(10)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4
5,ID_ec05b1a7b,180000.0,0,5,0,1,1,1,1.0,0,...,121,1444,16,121,4,1.777778,1.0,121.0,1444,4
6,ID_e9e0c1100,180000.0,0,5,0,1,1,1,1.0,0,...,4,64,16,121,4,1.777778,1.0,121.0,64,4
7,ID_3e04e571e,130000.0,1,2,0,1,1,0,NaN,0,...,0,49,16,81,4,16.000000,1.0,100.0,49,4
8,ID_1284f8aad,130000.0,1,2,0,1,1,0,NaN,0,...,81,900,16,81,4,16.000000,1.0,100.0,900,4
9,ID_51f52fdd2,130000.0,1,2,0,1,1,0,NaN,0,...,121,784,16,81,4,16.000000,1.0,100.0,784,4


#### Missing Data

In [8]:
missing_data=(all_data.isnull().sum()/len(all_data))
missing_data.sort_values(ascending=False).head(15)

rez_esc            0.825457
v18q1              0.762218
v2a1               0.726154
meaneduc           0.001077
SQBmeaned          0.001077
techozinc          0.000000
techoentrepiso     0.000000
techocane          0.000000
techootro          0.000000
cielorazo          0.000000
abastaguadentro    0.000000
sanitario3         0.000000
abastaguafuera     0.000000
abastaguano        0.000000
public             0.000000
dtype: float64

Above is a list of all the variables with missing data. Luckily, there's only 5. I will fill them in order. From the data description, rez_esc is the "years behind in school". I want to learn more about this variable before I decide how to deal with it, like if it's NaN if the person didn't attend school, or if it seems like it's NaN if the person is 0 years behind in school. 

In [9]:
print(np.mean(all_data["rez_esc"]))
print(np.std(all_data["rez_esc"]))
print(np.min(all_data["rez_esc"]))
print(np.max(all_data["rez_esc"]))

0.4348422496570645
1.5817481806773257
0.0
99.0


In [10]:
all_data[["rez_esc", "escolari"]].head(10)

,rez_esc,escolari
0,NaN,10
1,NaN,12
2,NaN,11
3,1.0,9
4,NaN,11
5,NaN,11
6,0.0,2
7,0.0,0
8,NaN,9
9,NaN,11


In [11]:
all_data["testing_rez"]=np.where(all_data["rez_esc"].isnull(), 1, 0)
print(all_data["testing_rez"].corr(all_data["escolari"]))
print(all_data["testing_rez"].corr(all_data["meaneduc"]))
print(pd.DataFrame.corrwith(all_data, all_data["testing_rez"]).sort_values(ascending=False).head(10))
print(pd.DataFrame.corrwith(all_data, all_data["rez_esc"]).sort_values(ascending=False).head())

0.21530368472725836
0.033162783793879984
testing_rez     1.000000
age             0.471585
SQBage          0.392844
agesq           0.392844
parentesco1     0.306266
estadocivil3    0.280629
SQBescolari     0.243302
parentesco2     0.217754
escolari        0.215304
instlevel8      0.187381
dtype: float64
rez_esc       1.000000
agesq         0.235885
SQBage        0.235885
age           0.224549
instlevel3    0.132524
dtype: float64


rez_esc is missing in 83% of the observations. It's not correlated with any of the education variables, and it's absense isn't correlated with any variables. It ranges from 0 to 99. Maybe I could fill it in with the average of the region, but that isn't highly correlated. Years of schooling has no missing data and is sufficient to cover anything that could be predicted with rez_esc, so I will just drop it. Next I will look to fill v18q1. "v18q1" is the number of tablets a household owns. 

In [12]:
all_data.drop(['rez_esc'], axis=1, inplace=True)

In [13]:
print(np.mean(all_data["v18q1"]))
print(np.std(all_data["v18q1"]))
print(np.min(all_data["v18q1"]))
print(np.max(all_data["v18q1"]))

1.364002517306482
0.7144384108611518
1.0
6.0


In [14]:
print(all_data[["v18q", "v18q1"]].head())
all_data["testv18q1"]=np.where(all_data['v18q1'].isnull(), 1, 0)
print(all_data[["testv18q1", "v18q"]].head())
print(all_data["testv18q1"].corr(all_data["v18q"]))
all_data.drop(['testv18q1'], axis=1, inplace=True)

   v18q  v18q1
0     0    NaN
1     1    1.0
2     0    NaN
3     1    1.0
4     1    1.0
   testv18q1  v18q
0          1     0
1          0     1
2          1     0
3          0     1
4          0     1
-0.9999999999999999


It's pretty clear that a null value for the number of tablets a family had means that the family has 0 tablets. First, the number of tablets a family has does not go down to zero already. Second, having a null value for the number of tablets a family has is nearly perfectly correlated with them saying that they did not have a tablet in the pervious question. So I'm just going to fill in the null values with zero. Then I will look at "v2a1", which is the variable for monthly rent payment, which has 73% of its values missing. 

In [15]:
all_data["v18q1"].fillna(0, inplace=True)

In [16]:
print(np.mean(all_data["v2a1"]), np.std(all_data["v2a1"]), np.max(all_data["v2a1"]), np.min(all_data["v2a1"]))

172030.8455737705 154995.03127164065 2852700.0 0.0


In [17]:
print(pd.DataFrame.corrwith(all_data, all_data["v2a1"]).sort_values(ascending=False).head(10))
print(pd.DataFrame.corrwith(all_data, all_data["v2a1"]).sort_values(ascending=True).head(10))
all_data["testv2a1"]=np.where(all_data["v2a1"].isnull(), 1, 0)
print(pd.DataFrame.corrwith(all_data, all_data["testv2a1"]).sort_values(ascending=False).head(10))
print(pd.DataFrame.corrwith(all_data, all_data["testv2a1"]).sort_values(ascending=True).head(10))

v2a1           1.000000
rooms          0.453555
meaneduc       0.401386
SQBmeaned      0.375273
bedrooms       0.344225
SQBedjefe      0.333262
v18q1          0.328233
SQBescolari    0.319282
cielorazo      0.301601
v18q           0.288995
dtype: float64
pisocemento       -0.258056
overcrowding      -0.240206
tipovivi3         -0.232605
epared2           -0.213151
eviv2             -0.207117
SQBovercrowding   -0.203074
energcocinar3     -0.189155
etecho2           -0.182297
area2             -0.178532
paredpreb         -0.158911
dtype: float64
testv2a1         1.000000
tipovivi1        0.790821
tipovivi5        0.186518
hogar_mayor      0.183686
area2            0.178311
agesq            0.168695
SQBage           0.168695
age              0.157075
elimbasu3        0.121721
energcocinar4    0.116172
dtype: float64
tipovivi3   -0.734760
tipovivi2   -0.556836
area1       -0.178311
meaneduc    -0.160948
elimbasu1   -0.153089
cielorazo   -0.141501
SQBmeaned   -0.140617
SQBedjefe   -0.139557

From the above correlations, it is clear that having a "NaN" value for monthly payments is highly correlated with owning a home, and having another (assigned or borrowed) home status. So all the people who own homes which are fully paid off can have a 0 for their monthly rent payment. Additionally, homes which are "assigned or borrowed", (which is also highly correlated with a missing rent value), can have 0. This deals with most of the missing variables in the v2a1 column. I'm going to go ahead and fill the remaining missing values in with the means from the region, because I believe it's practical to think that the remaining 2% of missing values are similar across regions. 

In [18]:
all_data["v2a1"]=np.where((all_data["v2a1"].isnull() & all_data["tipovivi1"]==1), 0, all_data["v2a1"])
all_data["v2a1"]=np.where((all_data["v2a1"].isnull() & all_data["tipovivi5"]==1), 0, all_data["v2a1"])

In [19]:
all_data["v2a1"].head()

0    190000.0
1    135000.0
2         0.0
3    180000.0
4    180000.0
Name: v2a1, dtype: float64

In [20]:
all_data["v2a1"] = all_data.groupby(("lugar1", "lugar2", "lugar3", "lugar4", "lugar5", "lugar6"))["v2a1"].transform(
    lambda x: x.fillna(x.median()))

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


Now that I've dealt with the missing values in v2a1, I will look at my last missing values: meaneduc and SQBmeaned, both of which have .1% of values missing. Because SQBmeaned is just the squareroot of meaneduc, I will fill in what I can for meaneduc and then fill the squareroots in for SQBmeaned. 

In [21]:
print(np.mean(all_data["meaneduc"]), np.std(all_data["meaneduc"]), np.max(all_data["meaneduc"]), np.min(all_data["meaneduc"]))

9.17866594238228 4.105663801258728 37.0 0.0


In [22]:
all_data["testmeaneduc"]=np.where(all_data["meaneduc"].isnull(), 1, 0)
print(pd.DataFrame.corrwith(all_data, all_data["testmeaneduc"]).sort_values(ascending=False).head(10))
print(pd.DataFrame.corrwith(all_data, all_data["testmeaneduc"]).sort_values(ascending=True).head(10))
all_data.drop(['testmeaneduc'], axis=1, inplace=True)


testmeaneduc     1.000000
SQBdependency    0.152053
elimbasu4        0.044101
sanitario5       0.039640
energcocinar1    0.038107
tipovivi4        0.030004
noelec           0.029514
paredmad         0.029389
tipovivi3        0.028983
sanitario1       0.028002
dtype: float64
hogar_adul    -0.074426
v14a          -0.064534
r4t2          -0.039733
r4t3          -0.038335
hogar_total   -0.038233
hhsize        -0.038233
tamhog        -0.038233
rooms         -0.036863
refrig        -0.035471
r4h3          -0.034837
dtype: float64


Missing data for meaneduc doesn't seemto be correlated with anything. It's only missing a few observations, so I'm going to fill null observations in with the mean of the region. 

In [23]:
all_data["meaneduc"] = all_data.groupby(("lugar1", "lugar2", "lugar3", "lugar4", "lugar5", "lugar6"))["meaneduc"].transform(
    lambda x: x.fillna(x.median()))

all_data["SQBmeaned"]=np.where(all_data["SQBmeaned"].isnull(), np.sqrt(all_data["meaneduc"]), all_data["SQBmeaned"])

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [24]:
missing_data=(all_data.isnull().sum()/len(all_data))
missing_data.sort_values(ascending=False).head(15)

testv2a1           0.0
testing_rez        0.0
sanitario5         0.0
sanitario3         0.0
sanitario2         0.0
sanitario1         0.0
coopele            0.0
noelec             0.0
planpri            0.0
public             0.0
abastaguano        0.0
abastaguafuera     0.0
abastaguadentro    0.0
cielorazo          0.0
techootro          0.0
dtype: float64

In [25]:
all_data.shape

(33413, 144)

## Feature Engineering 

Now I'm going to make sure all data has the correct type and generate new variables and generate variables to measure overall home quality, and scale variables by number of people in the household and age where appropriate. 
First, dependency is the number of dependents to working-aged people. It should be a float64 rather than an integer, so I am going to fill the "yes" and "no" with 1 and 0 respectively so I can make it the correct type. The same goes with edjefa and edjefe. The data description on Kaggle even says that Yes=1 and no=0 for these variables, so I feel comfortable making the substitutions. 

In [26]:
mapping = {"yes": 1, "no": 0}
all_data['dependency'] = all_data['dependency'].replace(mapping).astype(np.float64)
all_data['edjefa'] = all_data['edjefa'].replace(mapping).astype(np.float64)
all_data['edjefe'] = all_data['edjefe'].replace(mapping).astype(np.float64)


The difference in the number of people in a household and the household size may be indicative of poverty level because it can take into account overcrowding in the home. Additionally,  general variables describing the overall quality of the home aggregated up could be useful. 

In [27]:
all_data['hhsize-diff'] = all_data['tamviv'] - all_data['hhsize']

In [28]:
all_data['walls'] = np.argmax(np.array(all_data[['epared1', 'epared2', 'epared3']]), axis=1)
all_data['roof'] = np.argmax(np.array(all_data[['etecho1', 'etecho2', 'etecho3']]), axis=1)
all_data['floor'] = np.argmax(np.array(all_data[['eviv1', 'eviv2', 'eviv3']]), axis=1)
all_data['walls+roof+floor'] = all_data['walls'] + all_data['roof'] + all_data['floor']


Next I'm going to put in variables to measure overall quality of life from material goods and scale possessions by number of people in the home and by age of the individual.

In [29]:
all_data['nogood'] = 1 * (all_data['sanitario1'] +  
                         all_data['pisonotiene'] + 
                         all_data['abastaguano'] + 
                         (all_data['cielorazo'] == 0))
all_data['bonus'] = 1 * (all_data['refrig'] + 
                      all_data['computer'] + 
                      (all_data['v18q1'] > 0) + 
                      all_data['television'])


Household commodities should be scaled by the number of people in each household to provide the best measure of comparision across observations. 

In [30]:
all_data['phones-per-capita'] = all_data['qmobilephone'] / all_data['tamviv']
all_data['tablets-per-capita'] = all_data['v18q1'] / all_data['tamviv']
all_data['rooms-per-capita'] = all_data['rooms'] / all_data['tamviv']
all_data['rent-per-capita'] = all_data['v2a1'] / all_data['tamviv']
all_data['escolari/age'] = all_data['escolari'] / all_data['age']

In [31]:
all_data['tech'] = all_data['v18q'] + all_data['mobilephone']

Next I am going to make a dictioinary of variables which should be categorized as boolean, continuous, or ordered so I can ensure they are used properly.

In [32]:
id_ = ['Id', 'idhogar', 'Target']
hh_bool = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area1', 'area2']

hh_ordered = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
              'r4t3', 'v18q1', 'tamhog','tamviv','hhsize','hogar_nin','hhsize-diff',
              'walls', 'roof', 'floor', 'walls+roof+floor', 'nogood', 'bonus',
              'hogar_adul','hogar_mayor','hogar_total',  'bedrooms', 'qmobilephone']

hh_cont = ['v2a1', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding',
          'phones-per-capita', 'tablets-per-capita', 'rooms-per-capita', 'rent-per-capita']
ind_bool = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone']

ind_ordered = ['age', 'escolari',  'tech']

ind_cont = ['escolari/age']

In [33]:
for variable in (hh_bool + ind_bool):
    all_data[variable] = all_data[variable].astype('bool')
for variable in (hh_cont + ind_cont):
    all_data[variable] = all_data[variable].astype(float) 
for variable in (hh_ordered + ind_ordered):
        all_data[variable] = all_data[variable].astype(int)

In [34]:
all_data["parentesco1"].head()
print((all_data).shape)

(33413, 157)


I'm going to check and make sure that there is no more missing data after the feature engineering. 

In [35]:
missing_data=(all_data.isnull().sum()/len(all_data))
missing_data.sort_values(ascending=False).head(15)

escolari/age     0.010625
sanitario2       0.000000
energcocinar4    0.000000
energcocinar3    0.000000
energcocinar2    0.000000
energcocinar1    0.000000
sanitario6       0.000000
sanitario5       0.000000
sanitario3       0.000000
sanitario1       0.000000
elimbasu2        0.000000
coopele          0.000000
noelec           0.000000
planpri          0.000000
public           0.000000
dtype: float64

escolari/age has missing data, but that's from a divide by zero error. I'll go ahead and fill those with 0s. 

In [36]:
all_data["escolari/age"]=np.where(all_data["escolari/age"].isnull(), 0, all_data["escolari/age"])

In [37]:
missing_data=(all_data.isnull().sum()/len(all_data))
missing_data.sort_values(ascending=False).head(3)

tech             0.0
sanitario1       0.0
energcocinar3    0.0
dtype: float64

Because only the heads of household need to be accurately predicted, I'm only going to train on the heads of the household. This way I can maximize my predictive power on the correct targets. 

In [38]:
heads = all_data.loc[all_data['parentesco1'] == 1, :]
heads['Target'].head()



0    4
1    4
2    4
5    4
8    4
Name: Target, dtype: int64

In [39]:
heads.drop(['Id'], axis=1, inplace=True)
heads.drop(['idhogar'], axis=1, inplace=True)
heads=pd.get_dummies(heads)

C:\Users\owner\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [40]:
train = heads.loc[all_data['Target'] != 0, :]
test = all_data.loc[all_data['Target']==0, :]
Id=test['Id']
test.drop(['Id'], axis=1, inplace=True)
test.drop(['idhogar'], axis=1, inplace=True)
print(train.shape, test.shape)

(2973, 155) (23856, 155)


## Random Forest 

The first model I'm going to try will be a Random Forest because it's typically a very strong model and it will be a good benchmark to see how I'm doing with my data set. I will hypertune the parameters for number of trees, maximum depth, and class weight, (because there is not a uniform distribution of households across the different poverty classifications). 

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import mean_squared_error

In [42]:
target=train['Target']
train = train.drop('Target', axis=1)

In [43]:
x_train, x_test, y_train, y_test=train_test_split(train, target, test_size=.3)

In [44]:
from sklearn.model_selection import RandomizedSearchCV
param_dis= {"n_estimators": [70, 100, 120, 150], "max_depth": [5, 6, 8, 9], "class_weight": ["balanced", "balanced_subsample", None]}
clf=RandomForestClassifier()
n_inter=10
random_search=RandomizedSearchCV(clf, param_distributions=param_dis, n_iter=n_inter, cv=4, scoring='f1_macro')

random_search.fit(x_train, y_train)
print("Best Parameters: {}".format(random_search.best_params_))
print("Best Negative MSE: {}".format(random_search.best_score_))

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best Parameters: {'n_estimators': 100, 'max_depth': 8, 'class_weight': 'balanced_subsample'}
Best Negative MSE: 0.4158769832756503


It looks like the random forest model maximises the f1_macro score when it has 100 estimators, a max depth of 5, and uses a balanced subsample. It makes sense that the model results in a balanced subsample, because there was a huge inbalance in the classes, with most households being classfied as 4s and fewest being classified as 1s. I'm going to run the random forest model again, and this time use my tuned hyper parameters. 

In [46]:
clf=RandomForestClassifier(n_estimators=100, max_depth=8, class_weight="balanced_subsample")
clf.fit(x_train, y_train)
train_predictions = clf.predict(x_train)
test_predictions = clf.predict(x_test)

train_f1 = f1_score(y_train, train_predictions, average='macro')
test_f1 = f1_score(y_test, test_predictions, average='macro')

print("Train f1: {}".format(train_f1))
print("Test f1: {}".format(test_f1))

Train f1: 0.7835382006807614
Test f1: 0.43129996003197446


That's definitley overfitting, but the test f1 scoreis still relativley good, so I'm going to submit the model to kaggle and see how it goes. 

In [94]:
test = test.drop('Target', axis=1)


In [95]:
prediction_clf=clf.predict(test)

In [96]:
sub_clf=pd.DataFrame()
sub_clf['Id']=Id
sub_clf['target']=prediction_clf

In [97]:
sub_clf.to_csv("sub5_clf.csv", index=False)

The Kaggle score actually wasn't too bad. The f1_macro was 0.427. I'll have that be my personal benchmark for now and work with more models to see if I can fix the overfitting and get a better score.  

## Gradient Boosting 



The next model I will try is a Gradient Boosting Model. Gradient boosting has trees develop sequencially rather than independently, (like Random Forest), so this model seems to be a natural next step. I'll see if the sequencial development works better. 

In [98]:
from sklearn.ensemble import GradientBoostingClassifier

In [99]:
bgc=GradientBoostingClassifier()
param_dis= {"learning_rate": [.01, .1, 1], "n_estimators": [75, 100, 150], "tol": [.01, .1, .001]}
n_inter=10
random_search=RandomizedSearchCV(bgc, param_distributions=param_dis, n_iter=n_inter, cv=4, scoring='f1_macro')

random_search.fit(x_train, y_train)
print("Best Parameters: {}".format(random_search.best_params_))
print("Best Negative MSE: {}".format(random_search.best_score_))

Best Parameters: {'tol': 0.001, 'n_estimators': 150, 'learning_rate': 1}
Best Negative MSE: 0.3731007934512792


In [128]:
bgc=GradientBoostingClassifier(tol=0.001, n_estimators=100, learning_rate=1)
bgc.fit(x_train, y_train)
train_predictions = bgc.predict(x_train)
test_predictions = bgc.predict(x_test)

train_f1 = f1_score(y_train, train_predictions, average='macro')
test_f1 = f1_score(y_test, test_predictions, average='macro')

print("Train f1: {}".format(train_f1))
print("Test f1: {}".format(test_f1))

Train f1: 1.0
Test f1: 0.366903762859382


It looks like the model operates best with a tolerance of 0.001, 150 trees, and a learning rate of 1. While my last model was overfitting, this one is definitley overfitting worse. There is a huge bag betweenthe training and testing f1, and the training data is predicted perfectly. I could perhaps mitigate the overfitting by decreasing the number of trees, but even this optomized model does not do as well as the random forest, (as demonstrated below). 

In [143]:
prediction_BGC=bgc.predict(test)

In [144]:
sub_bgc=pd.DataFrame()
sub_bgc['Id']=Id
sub_bgc['target']=prediction_clf

In [146]:
sub_bgc.to_csv("sub1_bgc.csv", index=False)

This Kaggle score was 0.413, not as good as the Random Forest. So I'm going to continue with one final model.

## SVM

I chose to use SVM for my final model, which will trian a hypterplan to identify different classes. I chose this model because it has such a different approach than the decision trees I used in my previous two models. 

In [108]:
from sklearn.svm import SVC

In [129]:
svc=SVC()
gridsearch = GridSearchCV(svc, {"C": [0.05, 0.1, 0.01], "kernel": ['rbf'], 'gamma': [0.5, .75, .25]}, scoring='f1_macro')
gridsearch.fit(x_train, y_train)
print("Best Params: {}".format(gridsearch.best_params_))


C:\Users\owner\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Ana

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\c

Best Params: {'C': 0.05, 'gamma': 0.5, 'kernel': 'rbf'}


The best parameters for the model are to have a penalty of 0.05 and the gamma, or, Kernel coefficient for rbf as .5. I'll run the model again with those hyper parameter specifications. 

In [134]:
svc=SVC(C=.05, gamma=.5, kernel='rbf')
svc.fit(x_train, y_train)
train_predictions = svc.predict(x_train)
test_predictions = svc.predict(x_test)

train_f1 = f1_score(y_train, train_predictions, average='macro')
test_f1 = f1_score(y_test, test_predictions, average='macro')

print("Train f1: {}".format(train_f1))
print("Test f1: {}".format(test_f1))

Train f1: 0.19910352805089648
Test f1: 0.1963921034717495


C:\Users\owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [131]:
prediction_svc=svc.predict(test)

In [132]:
sub_svc=pd.DataFrame()
sub_svc['Id']=Id
sub_svc['target']=prediction_svc

In [133]:
sub_svc.to_csv("sub1_svc.csv", index=False)

This model still didn't do as well as the random forest model or the gradient boosting model. However, it doesn't overfit as bad. But with it's F1 score being less thant .2, it probably doesn't have signifigant predicting power. I will go forward with the Random Forest Model. 

## Conclusions

Ultimatley, my best model was the Random Forest. Furthar analysis on the usefulness of the model is below: 

In [48]:
clf.fit(x_train, y_train)
test_predictions = clf.predict(x_test)
print("Test Classification Report:")
print(classification_report(y_test, test_predictions))

Test Classification Report:
              precision    recall  f1-score   support

           1       0.34      0.32      0.33        69
           2       0.28      0.39      0.33       120
           3       0.20      0.26      0.23       108
           4       0.86      0.75      0.80       595

   micro avg       0.61      0.61      0.61       892
   macro avg       0.42      0.43      0.42       892
weighted avg       0.66      0.61      0.63       892



In [81]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',  ascending=False)
print(feature_importances.head(10))
feature_not_importances= pd.DataFrame(clf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',  ascending=True)
print(feature_not_importances.head(20))

                   importance
SQBmeaned            0.039992
meaneduc             0.037102
escolari             0.034236
escolari/age         0.032196
SQBescolari          0.027202
SQBage               0.026808
agesq                0.025768
phones-per-capita    0.025519
walls+roof+floor     0.024449
age                  0.023528
              importance
pisoother            0.0
parentesco8          0.0
parentesco6          0.0
parentesco9          0.0
parentesco10         0.0
parentesco5          0.0
parentesco11         0.0
parentesco12         0.0
estadocivil1         0.0
testing_rez          0.0
techootro            0.0
paredother           0.0
parentesco4          0.0
parentesco3          0.0
parentesco2          0.0
parentesco1          0.0
elimbasu5            0.0
elimbasu6            0.0
planpri              0.0
parentesco7          0.0


The model puts more weight on education, with education, mean education, and schooling scaled by age being the most heavily weighted variables. Variables like the predominant material on the floor/roof, family position, and rubbish disposal are not as important to the model. 


This model is far from perfect, but that does not mean it has no use. This model does a good job evaluating poverty levels for "level 4" households, with an f1_macro score of .80. It does not, however, do a good job at classifying homes in the 1, 2, or 3 category. For that reason, it could be used to initially screen out households which are level 4 for reciving certain types of aid, and because most homes are level 4 in the data set, more resources could be spent evaluating the differences between the needs of the homes remaining. That could save both time and money.